In [1]:
import pandas as pd
import numpy as np

In [2]:
bad_windows_file = '/project/fudenber_735/backup/DNN_HiC/akita_paper_data/akita_mm10/200213_gaps_mm10_binSize2048_numconseq10.bed'

In [3]:
bad_windows = pd.read_csv(bad_windows_file, sep="\t", names=["chr", "start", "end"])

In [4]:
def generate_good_windows(df_bad_windows):
    
    # Initialize a list to hold the good windows
    good_windows = []
    
    # Loop through the bad windows to create good windows
    for i in range(1, len(df_bad_windows)):
        prev_end = df_bad_windows.iloc[i-1]['end']
        curr_start = df_bad_windows.iloc[i]['start']
        
        # Create a good window if there's a gap
        if prev_end < curr_start:
            good_windows.append([df_bad_windows.iloc[i-1]['chr'], prev_end, curr_start])
    
    # Convert the good windows list to a DataFrame
    df_good = pd.DataFrame(good_windows, columns=['chr', 'start', 'end'])
    
    return df_good

In [5]:
# bad_windows

In [6]:
good_windows = generate_good_windows(bad_windows)

In [ ]:
# good_windows

In [7]:
chr1 = good_windows[good_windows["chr"] == "chr1"]

In [8]:
df = chr1[:2]

In [9]:
df

,chr,start,end
0,chr1,3000320,22433792
1,chr1,22466560,44365824


In [10]:
seq_length = 1310720
bin_size = 2048
split = 10

In [11]:
def create_expanded_df(df):
    # Initialize lists to store the resulting rows
    chr_list = []
    start_list = []
    end_list = []
    genome_window_start_list = []

    # Iterate through each good window
    for good_window in df.itertuples(index=False):
        num_jumps = (good_window.end - good_window.start - seq_length) // bin_size

        for bin_shift_index in range(num_jumps):
            genome_window_start = good_window.start + bin_shift_index * bin_size

            # Append the values to the respective lists
            chr_list.append(good_window.chr)
            start_list.append(good_window.start)
            end_list.append(good_window.end)
            genome_window_start_list.append(genome_window_start)
                
    # Create the resulting DataFrame
    expanded_df = pd.DataFrame({
        'chr': chr_list,
        'start': start_list,
        'end': end_list,
        'genome_window_start': genome_window_start_list,
    })

    return expanded_df

In [12]:
expanded_df = create_expanded_df(df)

In [13]:
expanded_df

,chr,start,end,genome_window_start
0,chr1,3000320,22433792,3000320
1,chr1,3000320,22433792,3002368
2,chr1,3000320,22433792,3004416
3,chr1,3000320,22433792,3006464
4,chr1,3000320,22433792,3008512
...,...,...,...,...
18897,chr1,22466560,44365824,43044864
18898,chr1,22466560,44365824,43046912
18899,chr1,22466560,44365824,43048960
18900,chr1,22466560,44365824,43051008


In [14]:
expanded_df.to_csv('chr1_2windows.tsv', sep="\t", index=False) 